## Setting up the COLAB runtime (user action required)

This colab-friendly notebook is targeted at demoing the enforcer on LLAMA2. It can run on a free GPU on Google Colab.
Make sure that your runtime is set to GPU:

Menu Bar -> Runtime -> Change runtime type -> T4 GPU (at the time of writing this notebook). [Guide here](https://www.codesansar.com/deep-learning/using-free-gpu-tpu-google-colab.htm).

## Gathering huggingface credentials (user action required)

We begin by installing the dependencies. This demo uses llama2, so you will have to create a free huggingface account, request access to the llama2 model, create an access token, and insert it when executing the next cell will request it.

Links:

- [Request access to llama model](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf). See the "Access Llama 2 on Hugging Face" section.
- [Create huggingface access token](https://huggingface.co/settings/tokens)


In [1]:
!pip install transformers torch lm-format-enforcer huggingface_hub accelerate bitsandbytes cpm_kernels
!huggingface-cli login

# When running from source / developing the library, use this instead
# %load_ext autoreload
# %autoreload 2
# import sys
# import os
# sys.path.append(os.path.abspath('..'))
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [2]:
import torch
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

model_id = 'meta-llama/Llama-2-7b-chat-hf'
device = 'cuda'

if torch.cuda.is_available():
    config = AutoConfig.from_pretrained(model_id)
    config.pretraining_tp = 1
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        config=config,
        torch_dtype=torch.float16,
        load_in_8bit=True,
        device_map='auto'
    )
else:
    raise Exception('GPU not available')
tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    # Required for batching example
    tokenizer.pad_token = tokenizer.eos_token  


/home/noamgat/mambaforge/envs/commentranker/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [03:44<00:00, 112.07s/it]
Using pad_token, but it is not set yet.


OK


If the previous cell executed successfully, you have propertly set up your Colab runtime and huggingface account!

## Setting up the prompt for the specific language model

We set up the prompting style according to the demo at https://huggingface.co/spaces/huggingface-projects/llama-2-13b-chat/blob/main/app.py . We simplify the implementation a bit as we don't need chat history for this demo.

In [3]:
def get_prompt(message: str, system_prompt: str) -> str:
    texts = [f'<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n']
    # The first user input is _not_ stripped
    do_strip = False
    message = message.strip() if do_strip else message
    texts.append(f'{message} [/INST]')
    return ''.join(texts)

## Calling generate_enforced() instead of model.generate()

The main function is fairly straightforward, except for the optional parameters ```required_regex``` / ```required_str``` / ```required_json_schema``` which activate the appropriate ```CharacterLevelParser``` to be used by the format enforcer.

The implementation includes ```output_scores=True``` when calling ```generate_enforced()```, which returns diagnostic information about the enforcer's actions. We will use this to understand the results later in this notebook.

In [4]:
%load_ext autoreload
%autoreload 2
from typing import Tuple, Optional, Union, List
import pandas as pd
from lmformatenforcer import JsonSchemaParser, CharacterLevelParser, RegexParser, StringParser, generate_enforced

StringOrManyStrings = Union[str, List[str]]

def run(message: StringOrManyStrings,
        system_prompt: str,
        max_new_tokens: int = 1024,
        temperature: float = 0.8,
        top_p: float = 0.95,
        top_k: int = 50,
        num_beams: int = 1,
        required_regex: Optional[str] = None,
        required_str: Optional[str] = None,
        required_json_schema: Optional[dict] = None) -> Tuple[StringOrManyStrings, Optional[pd.DataFrame]]:
    is_multi_message = isinstance(message, list)
    messages = message if is_multi_message else [message]
    prompts = [get_prompt(msg, system_prompt) for msg in messages]
    inputs = tokenizer(prompts, return_tensors='pt', add_special_tokens=False, return_token_type_ids=False, padding=is_multi_message).to(device)
    
    generate_kwargs = dict(
        inputs,
        # streamer=streamer,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_p=top_p,
        top_k=top_k,
        temperature=temperature,
        num_beams=num_beams,
        output_scores=True,
        return_dict_in_generate=True
    )

    parser: Optional[CharacterLevelParser] = None
    if required_regex:
        parser = RegexParser(required_regex)
    if required_str:
        parser = StringParser(required_str)
    if required_json_schema:
        parser = JsonSchemaParser(required_json_schema)

    if parser:
        output = generate_enforced(model, tokenizer, parser, **generate_kwargs)
    else:
        output = model.generate(**generate_kwargs)

    sequences = output['sequences']
    # skip_prompt=True doesn't work consistenly, so we hack around it.
    string_outputs = [tokenizer.decode(sequence, skip_special_tokens=True) for sequence in sequences]
    string_outputs = [string_output.replace(prompt[3:], ' ') for string_output, prompt in zip(string_outputs, prompts)]
    if parser and not is_multi_message:
        enforced_scores_dict = output.enforced_scores
        enforced_scores = pd.DataFrame(enforced_scores_dict)
        pd.set_option('display.width', 1000)
        pd.set_option('display.max_columns', 10)
        pd.set_option('display.max_rows', 999)
        pd.set_option('display.float_format', ' {:,.5f}'.format)
    else:
        enforced_scores = None
    return string_outputs if is_multi_message else string_outputs[0], enforced_scores


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## JSON Use case

Now we demnostrate using ```JsonSchemaParser```. We create a pydantic model, generate the schema from it, and use that to enforce the format.
The output will always be in a format that can be parsed by the parser.

In [6]:
from pydantic import BaseModel
from IPython.display import display, Markdown
from typing import List

def display_header(text):
    display(Markdown(f'**{text}**'))

def display_content(text):
    display(Markdown(f'```\n{text}\n```'))

DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\
"""
DEFAULT_MAX_NEW_TOKENS = 100

class AnswerFormat(BaseModel):
    first_name: str
    last_name: str
    year_of_birth: int
    num_seasons_in_nba: int

question = 'Please give me information about Michael Jordan. You MUST answer using the following json schema: '
question_with_schema = f'{question}{AnswerFormat.schema_json()}'

display_header("Question:")
display_content(question_with_schema)

display_header("Answer, With json schema enforcing:")
result, enforced_scores = run(question_with_schema, system_prompt=DEFAULT_SYSTEM_PROMPT, max_new_tokens=DEFAULT_MAX_NEW_TOKENS, required_json_schema=AnswerFormat.schema())
display_content(result)

display_header("Answer, Without json schema enforcing:")
result, _ = run(question_with_schema, system_prompt=DEFAULT_SYSTEM_PROMPT, max_new_tokens=DEFAULT_MAX_NEW_TOKENS)
display_content(result)



**Question:**

```
Please give me information about Michael Jordan. You MUST answer using the following json schema: {"title": "AnswerFormat", "type": "object", "properties": {"first_name": {"title": "First Name", "type": "string"}, "last_name": {"title": "Last Name", "type": "string"}, "year_of_birth": {"title": "Year Of Birth", "type": "integer"}, "num_seasons_in_nba": {"title": "Num Seasons In Nba", "type": "integer"}}, "required": ["first_name", "last_name", "year_of_birth", "num_seasons_in_nba"]}
```

**Answer, With json schema enforcing:**

```
   { "first_name": "Michael", "last_name": "Jordan", "year_of_birth": 1963, "num_seasons_in_nba": 15 }
```

**Answer, Without json schema enforcing:**

```
   Of course, I'd be happy to provide information about Michael Jordan using the provided JSON schema. Here's the response:
{
"title": "AnswerFormat",
"type": "object",
"properties": {
"first_name": {
"title": "First Name",
"type": "string",
"required": true

},
"last_name": {

"title": "Last Name",

"type":
```

## Understanding the results
Both runs used the exact same prompt, but only the enforced run produced a valid JSON output.
How did the enforcer cause the output to conform to the format? Lets look at the enforcer intervention table:


In [ ]:
display_header("Enforcer intervention table")
display(enforced_scores)

**Enforcer intervention table**

,generated_token,generated_token_idx,generated_score,leading_token,leading_token_idx,leading_score
0,▁,29871,0.99999,▁,29871,0.99999
1,▁{,426,0.00001,▁Of,4587,0.99382
2,"▁""",376,0.00312,<0x0A>,13,0.99674
3,first,4102,0.00022,title,3257,0.99825
4,_,29918,0.99994,_,29918,0.99994
5,name,978,1.00000,name,978,1.00000
6,""":",1115,0.99986,""":",1115,0.99986
7,"▁""",376,0.99994,"▁""",376,0.99994
8,Michael,24083,0.99161,Michael,24083,0.99161
9,""",",613,0.98919,""",",613,0.98919


Token index 3 shows where the enforcer had to be aggressive:


```
generated_token  generated_token_idx  generated_score leading_token  leading_token_idx  leading_score

3            first                 4102         0.00032         title               3257       0.94433
```

The language model was trying to generate the word "title" (post softmax score of 0.94433) but the enforcer made the "first" token (post softmax score of 0.00032) be the main candidate instead.
This can be used to further improve the prompt engineering, as we generally want to avoid timesteps that cause the enforcer to be this aggressive, as it increases the likelyhood of hallucinations. For example, The [langchain project removes the "title" from the json schema](https://github.com/langchain-ai/langchain/blob/cfa2203c626a2287d60c1febeb3e3a68b77acd77/libs/langchain/langchain/output_parsers/pydantic.py#L40), probably for this reason.

## Non-JSON Use Cases
Here are a few examples for the simple (non-json) case. Note that the ```required_regex``` does not support the full regex syntax, see the README for full limiations.

In [ ]:
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\
"""
MAX_MAX_NEW_TOKENS = 200
DEFAULT_MAX_NEW_TOKENS = 100
MAX_INPUT_TOKEN_LENGTH = 4000

integer_regex = ' Michael Jordan was born in (\d)+.'
question = 'In what year was Michael Jordan Born? Please answer using only a number, without any prefix or suffix.'
display_header("Question:")
display_content(question)

display_header("Without enforcing:")
result, _ = run(question, system_prompt=DEFAULT_SYSTEM_PROMPT, max_new_tokens=DEFAULT_MAX_NEW_TOKENS)
display_content(result)

print('\n----------------------------------------\n')

display_header(f"With regex force. Regex: {integer_regex}")
result, enforced_scores = run(question, system_prompt=DEFAULT_SYSTEM_PROMPT, max_new_tokens=DEFAULT_MAX_NEW_TOKENS, required_regex=integer_regex)
display_header("Language model output:")
display_content(result)
display_header("Enforcer intervention table:")
display(enforced_scores)

print('\n----------------------------------------\n')

display_header("With string force:")
result, enforced_scores = run(question, system_prompt=DEFAULT_SYSTEM_PROMPT, max_new_tokens=DEFAULT_MAX_NEW_TOKENS, required_str='The answer is 1963')
display_header("Language model output:")
display_content(result)
display_header("Enforcer intervention table:")
display(enforced_scores)

**Question:**

```
In what year was Michael Jordan Born? Please answer using only a number, without any prefix or suffix.
```

**Without enforcing:**

```
   Thank you for your question! Michael Jordan was born in the year 1963.
```


----------------------------------------



...

**With string force:**

**Language model output:**

```
 The answer is 1963
```

**Enforcer intervention table:**

,generated_token,generated_token_idx,generated_score,leading_token,leading_token_idx,leading_score
0,The,1576,0.00000,▁,29871,0.99998
1,▁answer,1234,0.23457,▁question,1139,0.26999
2,▁is,338,0.03131,▁to,304,0.92951
3,▁,29871,0.59562,▁,29871,0.59562
4,1,29896,0.97437,1,29896,0.97437
5,9,29929,0.99911,9,29929,0.99911
6,6,29953,0.99843,6,29953,0.99843
7,3,29941,0.99771,3,29941,0.99771
8,</s>,2,0.00544,.,29889,0.98834


## Batching example

This is a simple example of using batching to generate multiple queries in parallel. All outputs will be in the correct format. Every timestep can filter different tokens for the different batch indices.

In [ ]:
PLAYER_NAMES = ['Michael Jordan', 'Tim Duncan', 'Kobe Bryant', 'Kareem Abdul Jabbar']
question = 'Please give me information about {0}. You MUST answer using the following json schema: '
questions_with_schema = [f'{question.format(player_name)}{AnswerFormat.schema_json()}' for player_name in PLAYER_NAMES]

display_header("Batched Answers, With json schema enforcing:")
results, _ = run(questions_with_schema, system_prompt=DEFAULT_SYSTEM_PROMPT, max_new_tokens=DEFAULT_MAX_NEW_TOKENS, required_json_schema=AnswerFormat.schema())
for result in results:
    display_content(result)

**Batched Answers, With json schema enforcing:**

```
   { "first_name": "Michael", "last_name": "Jordan", "year_of_birth": 1963, "num_seasons_in_nba": 15 }
```

```
   { "first_name": "Timothy", "last_name": "Duncan", "year_of_birth": 1976, "num_seasons_in_nba": 19 }
```

```
   { "first_name": "Kobe", "last_name": "Bryant", "year_of_birth": 1978, "num_seasons_in_nba": 20 }
```

```
   { "first_name": "Kareem", "last_name": "Abdul-Jabbar", "year_of_birth": 1947, "num_seasons_in_nba": 20 }
```